# Класс Tensor

In [1]:
import tensorflow as tf
# import tensorflow.compat.v2 as tf

In [2]:
# Определение входного тензора
x = [[-3], [8]]
# Определение операций и вычисление 
w = tf.Variable(initial_value=[[1, 1], [1, 1]], dtype=tf.float32, shape=(2, 2))
b = tf.constant(value=[[1], [1]], dtype=tf.float32, shape=(2, 1))
# Вычисления
y = tf.add(tf.matmul(w, x), b)
print(y)
w.assign(value=[[-1, -1], [-1, -1]])
y = tf.add(tf.matmul(w, x), b)
print(y)

tf.Tensor(
[[6.]
 [6.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[-4.]
 [-4.]], shape=(2, 1), dtype=float32)


# Нейронная сеть через Sequential

Загружаем набор данных

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Создаём Sequential, задающую структуру сети

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Задаём функцию потерь

In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Конфигурируем модель

In [6]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Обучаем модель

In [7]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2951 - accuracy: 0.9150
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1431 - accuracy: 0.9578
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1082 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0893 - accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0779 - accuracy: 0.9761


Оцениваем качество модели

In [8]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0817 - accuracy: 0.9736 - 408ms/epoch - 1ms/step


[0.08165188878774643, 0.9735999703407288]

# Нейорнная сеть через класс

Загружаем набор данных

In [9]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

Перемешиваем данные и организуем в батчи

In [10]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Создаём модель

In [11]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
    self.flatten = tf.keras.layers.Flatten()
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Выбираем функцию потерь и оптимизатор для обучения и создаём для них объекты

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Выбираем метрики для оценки суммарной ошибки и качества по эпохам

In [13]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Метод для обучения модели

In [14]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Метод для тестирования модели

In [15]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

Запуск обучения и оценки

In [16]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13563024997711182, Accuracy: 95.91000366210938, Test Loss: 0.06432738900184631, Test Accuracy: 97.79999542236328
Epoch 2, Loss: 0.04168655350804329, Accuracy: 98.7249984741211, Test Loss: 0.052214235067367554, Test Accuracy: 98.3699951171875
Epoch 3, Loss: 0.021456176415085793, Accuracy: 99.30833435058594, Test Loss: 0.058210209012031555, Test Accuracy: 98.31999969482422
Epoch 4, Loss: 0.01269690040498972, Accuracy: 99.5566635131836, Test Loss: 0.05921275541186333, Test Accuracy: 98.4000015258789
Epoch 5, Loss: 0.00940917618572712, Accuracy: 99.69166564941406, Test Loss: 0.06137390807271004, Test Accuracy: 98.43000030517578
